# Importing necessary Libraries

In [55]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType

In [56]:
KEY = 'cba050788d4e4e69aa869ee84cb6c9e6'
ENDPOINT = 'https://facialrecognition244.cognitiveservices.azure.com/'

In [57]:
# import modules
import requests
import cv2
# Request headers set Subscription key which provides access to this API. Found in
# your Cognitive Services accounts.
headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': KEY,
}
body = dict()
img_url = "http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659"
img_url2 = "https://www.stockphotosecrets.com/wp-content/uploads/2016/05/GettyImages-476996143.jpg"
body["url"] = "http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659"
body = str(body)
# Request URL
FaceApiDetect = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/detect?returnFaceId=true&returnFaceAttributes=age,gender,headPose,smile,facialHair'
try:
    # REST Call
    response = requests.post(FaceApiDetect, data=body, headers=headers)
    print("RESPONSE:" + str(response.json()))
except Exception as e:
    print(e)

RESPONSE:[{'faceId': '8cfe04ac-3917-436d-9eff-af1096bedccb', 'faceRectangle': {'top': 335, 'left': 249, 'width': 319, 'height': 319}, 'faceAttributes': {'smile': 1.0, 'headPose': {'pitch': -6.9, 'roll': -4.8, 'yaw': -0.8}, 'gender': 'male', 'age': 39.0, 'facialHair': {'moustache': 0.1, 'beard': 0.1, 'sideburns': 0.1}}}]


In [58]:
response = response.json()
print(response)

[{'faceId': '8cfe04ac-3917-436d-9eff-af1096bedccb', 'faceRectangle': {'top': 335, 'left': 249, 'width': 319, 'height': 319}, 'faceAttributes': {'smile': 1.0, 'headPose': {'pitch': -6.9, 'roll': -4.8, 'yaw': -0.8}, 'gender': 'male', 'age': 39.0, 'facialHair': {'moustache': 0.1, 'beard': 0.1, 'sideburns': 0.1}}}]


In [59]:
faceId = response[0]["faceId"]
print(faceId)
faceRectangle = response[0]["faceRectangle"]
print(faceRectangle)
faceAttributes = response[0]["faceAttributes"]
gender = faceAttributes['gender']
print(gender)
smile = faceAttributes['smile']
print(smile)
age = faceAttributes['age']
print(age)

8cfe04ac-3917-436d-9eff-af1096bedccb
{'top': 335, 'left': 249, 'width': 319, 'height': 319}
male
1.0
39.0


In [60]:
# Checking if there exist multiples facesa or single face. 

In [61]:
if len(response) > 1:
    print("Multiple faces Exist")
else: 
    print("Only single face found")

Only single face found


# Converting Image object into numpy array.

In [62]:
import numpy as np

response = requests.get(img_url)
img = Image.open(BytesIO(response.content))
img = np.asarray(img)

# plotting rectangle against detected face in the image.

In [63]:
# faceRectangle["left"]
x = faceRectangle["left"]
y = faceRectangle["top"]
w = faceRectangle["width"]
h = faceRectangle["height"]


cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)  
cv2.imshow('Pic with squares',img)
cv2.waitKey(0) 
cv2.destroyAllWindows()


# Here we are creating group in which we will store images of different person 

In [66]:
personGroupId = "friends"
body = dict()
body["name"] = "F.R.I.E.N.D.S"
body["userData"] = "All friends cast"
body = str(body)
#Request URL
FaceApiCreateLargePersonGroup = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId
try:
    # REST Call
    response = requests.put(FaceApiCreateLargePersonGroup, data=body, headers=headers)
    print("RESPONSE:" + str(response.status_code))
except Exception as e:
    print(e)

RESPONSE:409


In [65]:
print(body)

{'name': 'F.R.I.E.N.D.S', 'userData': 'All friends cast'}


# Here we are storing the meta data of persons such as name of a person, gender, age etc....

In [67]:
# Request Body
body = dict()
body["name"] = "Chandler"
body["userData"] = "Friends"
body = str(body)
# Request URL
FaceApiCreatePerson = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/' + personGroupId + '/persons'
try:
    # REST Call
    response = requests.post(FaceApiCreatePerson, data=body, headers=headers)
    responseJson = response.json()
    personId = responseJson["personId"]
    print("PERSONID: " + str(personId))
except Exception as e:
    print(e)

PERSONID: bcda18cd-873f-48e9-8b8c-14d90998a160


# Providing multiples images of different persons
### We are storing 5 images of each person in the personGroups so that later on we can train a model on the detected faces.


In [79]:
# 5 random images of chandler
chandlerImageList = ["http://www.imagozone.com/var/albums/vedete/Matthew%20Perry/Matthew%20Perry.jpg?m=1355670659",
"https://i.pinimg.com/236x/b0/57/ff/b057ff0d16bd5205e4d3142e10f03394--muriel-matthew-perry.jpg",
"https://qph.fs.quoracdn.net/main-qimg-74677a162a39c79d6a9aa2b11cc195b1",
"https://pbs.twimg.com/profile_images/2991381736/e2160154f215a325b0fc73f866039311_400x400.jpeg",
"https://i.pinimg.com/236x/f2/9f/45/f29f45049768ddf5c5d75ff37ffbfb3f--hottest-actors-matthew-perry.jpg"]

joeyImageList = [
   "https://upload.wikimedia.org/wikipedia/en/d/da/Matt_LeBlanc_as_Joey_Tribbiani.jpg",
    "https://a1cf74336522e87f135f-2f21ace9a6cf0052456644b80fa06d4f.ssl.cf2.rackcdn.com/images/characters_opt/p-friends-matt-leblanc.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/X_217c5312.jpg/220px-X_217c5312.jpg",
    "https://media3.s-nbcnews.com/j/newscms/2018_26/1349761/matt-leblanc-joey-friends-today-180629-tease_e00b0cfeb43333656f5334f3ea892fe8.social_share_1200x630_center.jpg",
    "https://i.pinimg.com/originals/d0/61/02/d061022a20400c1e2db1ce197bc8e64a.jpg"
]

# Request URL
FaceApiCreatePerson = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/' + personGroupId + '/persons/' + personId + '/persistedFaces'

for image in chandlerImageList:
    body = dict()
    body["url"] = image
    body = str(body)
    try:
        # REST Call
        response = requests.post(FaceApiCreatePerson, data=body, headers=headers)
        responseJson = response.json()
        persistedFaceId = responseJson["persistedFaceId"]
        print("PERSISTED FACE ID: " + str(persistedFaceId))
    except Exception as e:
        print(e)

HTTPSConnectionPool(host='facialrecognition244.cognitiveservices.azure.com', port=443): Max retries exceeded with url: /face/v1.0/persongroups/friends/persons/bcda18cd-873f-48e9-8b8c-14d90998a160/persistedFaces (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fb4a50020d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
HTTPSConnectionPool(host='facialrecognition244.cognitiveservices.azure.com', port=443): Max retries exceeded with url: /face/v1.0/persongroups/friends/persons/bcda18cd-873f-48e9-8b8c-14d90998a160/persistedFaces (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fb4a5002f70>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
HTTPSConnectionPool(host='facialrecognition244.cognitiveservices.azure.com', port=443): Max retries exceeded with url: /face/v1.0/persongroups/friends/persons/bcda18cd-873f-48e9-8b8c-14d90998a160/

# Training
### Here we are doing training on provided face images

In [73]:
#Request Body
body = dict()

#Request URL 
FaceApiTrain = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId+'/train'

try:
    # REST Call 
    response = requests.post(FaceApiTrain, data=body, headers=headers) 
    print("RESPONSE:" + str(response.status_code))

except Exception as e:
    print(e)

RESPONSE:202


# Detect face 
### Here We are passing the image to get the face coordinates. 

In [75]:
# Request Body
body = dict()
body["url"] = "https://vignette.wikia.nocookie.net/friends/images/9/99/Season_2_chandler.png/revision/latest/scale-to-width-down/340?cb=20200326151017"
body = str(body)

# Request URL 
FaceApiDetect = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/detect?returnFaceId=true' 

try:
# REST Call 
    response = requests.post(FaceApiDetect, data=body, headers=headers) 
    responseJson = response.json()
    faceId = responseJson[0]["faceId"]
    print("FACE ID: "+str(faceId))

except Exception as e:
    print(e)

FACE ID: 51392e07-3b8f-4ba3-9a96-e0b836e0c534


# Recognizing face

### Now here we will compute similarity score between the provided face and the stored faces in person group and the face that matches with the queried face will return its ID and a similarity score.

In [76]:
faceIdsList = [faceId]

# Request Body
body = dict()
body["personGroupId"] = personGroupId
body["faceIds"] = faceIdsList
body["maxNumOfCandidatesReturned"] = 1 
body["confidenceThreshold"] = 0.5
body = str(body)

# Request URL 
FaceApiIdentify = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/identify' 

try:
    # REST Call 
    response = requests.post(FaceApiIdentify, data=body, headers=headers) 
    responseJson = response.json()
    personId = responseJson[0]["candidates"][0]["personId"]
    confidence = responseJson[0]["candidates"][0]["confidence"]
    print("PERSON ID: "+str(personId)+ ", CONFIDENCE :"+str(confidence))
        
except Exception as e:
    print("Could not detect")

PERSON ID: bcda18cd-873f-48e9-8b8c-14d90998a160, CONFIDENCE :0.65836


# Finally, retrieve a person’s name and userData representing the registered person face image.

In [77]:
# Request URL 
FaceApiGetPerson = 'https://facialrecognition244.cognitiveservices.azure.com/face/v1.0/persongroups/'+personGroupId+'/persons/'+personId

try:
    response = requests.get(FaceApiGetPerson, headers=headers) 
    responseJson = response.json()
    print("This Is "+str(responseJson["name"]))
    
except Exception as e:
    print(e)

This Is Chandler


### Meta data stored against the person Id

In [78]:
responseJson

{'personId': 'bcda18cd-873f-48e9-8b8c-14d90998a160',
 'persistedFaceIds': ['7205c8fc-0a0a-4dbe-8530-7339ebbc28c6',
  '7406e56f-11ae-4056-a58a-321ae68b8297',
  '96dda81f-9a6b-474c-adaa-645c83ba26e3',
  'c840dafb-c794-41c6-b629-0b302d1b8020',
  'fc180b57-75c3-4af9-86d5-ad0c9d71f121'],
 'name': 'Chandler',
 'userData': 'Friends'}

# THE END HERE
.....................................................................................

# Testing New Code

In [29]:
from PIL import ImageFont

def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left, top), (right, bottom))

In [32]:
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))
detected_faces = face_client.face.detect_with_url(url=img_url2)

# Download the image from the url
response = requests.get(img_url2)
img = Image.open(BytesIO(response.content))

# For each face returned use the face rectangle and draw a red box.
print('Drawing rectangle around face... see popup for results.')
draw = ImageDraw.Draw(img)


for face in detected_faces:
    font = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 20)
    draw.text((50, 0), "Helo", (255, 255, 255), font=font)
    draw.rectangle(getRectangle(face), outline='red',  width=10)
    
img.show()

Drawing rectangle around face... see popup for results.
